## Группа DS03-onl

Студент Парфимович Алексей

## Домашнее задание №28

#### Оценить возраст человека по изображению. 
В качестве фьючеэкстрактора выбрать любую современную нейронную сеть. Задачу можно решать как задачу регрессии или классификации.
Если есть время - попробовать оба способа. Попробовать разблокировать часть слоев после обучения и дообучить модель.

В качестве датасета взять датасет лиц по ссылке 
https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/wiki_crop.tar
(описание - https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/ )

Примечания:
- Для решения задачи регресси использовать flow_from_dataframe
- Сначала создать датафрейм содержащий относительный путь к изображению и целевую метку, который потом использовать при обучении

In [12]:
import os
import sys
import wget
import random
import tarfile

import re
import dlib
import numpy as np
import pandas as pd

from PIL import Image

%matplotlib inline
from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow import keras
from keras.utils import to_categorical

from sklearn.utils import class_weight
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix, classification_report

DATA_URL = 'https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/wiki_crop.tar'
DATA_FILE = 'wiki_crop.tar'
DATA_FOLDER = 'wiki_crop'
RANDOM_STATE = 42

print('Cuda version: ' + tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Cuda version: 2.10.1
Num GPUs Available:  1


Загрузим файл с данными по ссылке (если файл не найден на диске)

In [13]:
def bar_custom(current, total, width=80):
    progress_msg = "Downloading: %d%% [%d / %d] bytes" % (current / total * 100, current, total)
    sys.stdout.write('\r' + progress_msg)
    sys.stdout.flush()

if not os.path.exists(DATA_FILE):
    wget.download(DATA_URL, DATA_FILE, bar=bar_custom)

Downloading: 100% [811315200 / 811315200] bytes

Распакуем архив с набором данных в текущий каталог,
если каталог с данными еще не существует.

In [3]:
if not os.path.exists(DATA_FOLDER):
    tar = tarfile.open(DATA_FILE, 'r')
    tar.extractall()
    tar.close()

Выполним загрузку, проверку и классификацию набора изображений из файлов

In [4]:
face_detector = dlib.get_frontal_face_detector()

# Набор всех корректных изображений для оценки возраста
images = pd.DataFrame(columns=['file_path','label'])

img_valid=0 #счетчик всех корректных изображений
img_misloaded=0 #счетчик изображений с ошибками при загрузке
img_missized=0 #счетчик изображений некорректного размера (1х1)
img_wo_faces=0 #счетчик изображений без лиц
img_misclassified=0 #счетчик изображений с некорретной оценкой класса

# Перебор всех вложенных элементов в указаном каталоге
for dir in os.listdir(DATA_FOLDER):
    # если вложенный элемент - каталог
    if os.path.isdir(os.path.join(DATA_FOLDER, dir)):
        # Перебор всех файлов в каталоге
        for file in os.listdir(os.path.join(DATA_FOLDER, dir)):
            try:
                file_path = os.path.join(DATA_FOLDER, dir, file)

                # загрузить изображение
                img = Image.open(file_path) #.convert('RGB')

                # получить параметры изображения
                h, w, c = np.array(img).shape

                # если размер изображения не корректен - пропускаем
                if h==w==1:
                    img_missized+=1 
                    continue

                # конвертируем изображение в монохромное для проверки детектирования лиц
                img_gray = np.array(img.convert('L'))
                face_rects = face_detector(img_gray, 0)

                # если на изображении не получается детектировать лица - пропускаем
                if len(face_rects) == 0:
                    img_wo_faces+=1
                    continue

                year_matched = re.findall(r'_(\d{4})', file)
                try:
                    label = int(year_matched[1]) - int(year_matched[0])
                except:
                    label = -1

                images = images.append({'file_path':file_path, 'label':label}, ignore_index=True)

                #if label<=0 or label>100:
                #    img_misclassified+=1
                #    continue              
                #img_valid+=1

            except Exception as ex:
                img_misloaded+=1
                None # Если файл не удается прочитать - пропускаем его

print(f'Изображений с ошибками при загрузке {img_misloaded}')
print(f'Изображений с некорректными размерами {img_missized}')
print(f'Изображений на которых не найдены лица {img_wo_faces}')

print(f'Всего корректных изображений {images.shape[0]}')

Изображений с ошибками при загрузке 12996
Изображений с некорректными размерами 0
Изображений на которых не найдены лица 19065
Всего корректных изображений 30267


Посмотрим что получилось:

In [5]:
images.describe()

,file_path,label
count,30267,30267
unique,30267,119
top,wiki_crop\00\10049200_1891-09-16_1958.jpg,26
freq,1,1191


Проверим набор данных на наличие некоректных меток предполагаемого возраста (<= 3 лет или > 100)

In [6]:
images[(images.label <=3) | (images.label > 100)].value_counts()

file_path                                  label
wiki_crop\01\658701_1712-06-14_1962.jpg     250     1
wiki_crop\52\9959752_1897-12-24_2006.jpg    109     1
wiki_crop\62\491662_1863-04-03_2004.jpg     141     1
wiki_crop\64\483964_1893-01-31_2001.jpg     108     1
wiki_crop\67\9304667_1896-09-21_2010.jpg    114     1
wiki_crop\68\23555068_1895-08-03_2005.jpg   110     1
wiki_crop\68\25377668_1760-05-05_1945.jpg   185     1
wiki_crop\69\3139869_1743-09-11_2012.jpg    269     1
wiki_crop\73\23278173_1897-04-19_2012.jpg   115     1
wiki_crop\75\32659275_1899-08-14_2009.jpg   110     1
wiki_crop\76\34011376_1898-09-07_2011.jpg   113     1
wiki_crop\78\11171378_1887-03-20_2012.jpg   125     1
wiki_crop\79\16127179_1896-11-13_2007.jpg   111     1
wiki_crop\84\24383784_1910-03-27_2011.jpg   101     1
wiki_crop\90\1202690_1878-01-21_2003.jpg    125     1
wiki_crop\90\9271090_1896-10-13_2007.jpg    111     1
wiki_crop\92\14449892_1894-08-08_2002.jpg   108     1
wiki_crop\92\42009092_2014-02-20_

Удалим записи с некорректными метками

In [7]:
df = images

df.drop(df[(df.label <=3) | (df.label > 100)].index, inplace=True)

Выполним секвестр классов согласно возрастной классификации американской ассоциации здравоохранения:  
    - до 12 лет - Детство (0)  
    - до 16 лет - Отрочество (1)  
    - до 21 лет - Юность (2)  
    - до 35 лет - Молодость (3)  
    - до 60 лет - Зрелость (4)  
    - свыше 60  - Старость (5)  

In [8]:
df.loc[df.label <= 12 , 'label'] = 0 
df.loc[(df.label > 12) & (df.label <= 16) , 'label'] = 1 
df.loc[(df.label > 16) & (df.label <= 21) , 'label'] = 2
df.loc[(df.label > 21) & (df.label <= 35) , 'label'] = 3
df.loc[(df.label > 35) & (df.label <= 60) , 'label'] = 4
df.loc[(df.label > 60), 'label'] = 5   

df.label.value_counts()

3    13309
4    10390
5     3965
2     2293
1      215
0       54
Name: label, dtype: int64

Сохранить результат обработки данных в CSV-файл

In [9]:
df.to_csv('wiki_crop.csv')